<a href="https://colab.research.google.com/github/lsteffenel/CHPS0906/blob/main/TP1/5-Visualiser%20une%20convolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Couche convolutionnelle

Dans ce notebook nous souhaitons visualiser quatre sorties filtrées (également appelées cartes d'activation) d'une couche convolutionnelle.

Pour cela, *nous* définissons quatre filtres qui sont appliqués à une image d'entrée en initialisant les **poids** d'une couche convolutionnelle.

Bien qu'amusant, on n'a pas à faire ça. En effet, un CNN entraîné apprendra les valeurs de ces poids tout seul, sans besoin qu'on initialise les filtres.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/conv-visualization/notebook_ims/conv_layer.gif?raw=1' height=60% width=60% />

### Importer une image

In [ ]:
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# TODO: Déposer une image dans l'onglet "Fichiers". Ensuite, renseignez le nom du fichier ci-dessous
img_path = 'car_sdc.png'

# load color image
bgr_img = cv2.imread(img_path)
# convert to grayscale
gray_img = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2GRAY)

# normalize, rescale entries to lie in [0,1]
gray_img = gray_img.astype("float32")/255

# plot image
plt.imshow(gray_img, cmap='gray')
plt.show()

### Définir des filtres

In [ ]:
import numpy as np

## TODO: Vous pouvez essayer autres filtres
filter_vals = np.array([[-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1], [-1, -1, 1, 1]])

print('Filter shape: ', filter_vals.shape)


In [ ]:
# Defining four different filters,
# all of which are linear combinations of the `filter_vals` defined above

# define four filters
filter_1 = filter_vals
filter_2 = -filter_1
filter_3 = filter_1.T
filter_4 = -filter_3
filters = np.array([filter_1, filter_2, filter_3, filter_4])

# For an example, print out the values of filter 1
print('Filter 1: \n', filter_1)

In [ ]:
# visualize all four filters
fig = plt.figure(figsize=(10, 5))
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))
    width, height = filters[i].shape
    for x in range(width):
        for y in range(height):
            ax.annotate(str(filters[i][x][y]), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if filters[i][x][y]<0 else 'black')

## Définir une couche convolutionnelle

Les différentes couches qui composent tout réseau neuronal sont documentées, [ici](http://pytorch.org/docs/stable/nn.html). Pour un réseau neuronal convolutionnel, nous commencerons par définir :
* Couche convolutionnelle

Initialisez une seule couche convolutionnelle afin qu'elle contienne tous vos filtres créés. Notez que vous n'entraînez pas ce réseau ; vous initialisez les poids dans une couche convolutionnelle afin de pouvoir visualiser ce qui se passe après un passage en avant à travers ce réseau !

#### `__init__` et `forward`
Pour définir un réseau neuronal dans PyTorch, vous définissez les couches d'un modèle dans la fonction `__init__` et définissez le comportement en avant d'un réseau qui applique ces couches initialisées à une entrée (`x`) dans la fonction `forward`. Dans PyTorch, nous convertissons toutes les entrées dans le type de données Tensor, qui est similaire à un type de données de liste en Python.

Ci-dessous, la structure d'une classe appelée `Net` possède une couche convolutionnelle pouvant contenir quatre filtres en niveaux de gris 4x4.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# define a neural network with a single convolutional layer with four filters
class Net(nn.Module):

    def __init__(self, weight):
        super(Net, self).__init__()
        # initializes the weights of the convolutional layer to be the weights of the 4 defined filters
        k_height, k_width = weight.shape[2:]
        # assumes there are 4 grayscale filters
        self.conv = nn.Conv2d(1, 4, kernel_size=(k_height, k_width), bias=False)
        self.conv.weight = torch.nn.Parameter(weight)

    def forward(self, x):
        # calculates the output of a convolutional layer
        # pre- and post-activation
        conv_x = self.conv(x)
        activated_x = F.relu(conv_x)

        # returns both layers
        return conv_x, activated_x

# instantiate the model and set the weights
weight = torch.from_numpy(filters).unsqueeze(1).type(torch.FloatTensor)
model = Net(weight)

# print out the layer in the network
print(model)

### Visualiser la sortie de chaque filtre

Nous allons d'abord définir une fonction d'assistance, `viz_layer`, qui prend en compte une couche spécifique et un nombre de filtres (argument facultatif) et affiche la sortie de cette couche une fois qu'une image a été transmise.

In [ ]:
# helper function for visualizing the output of a given layer
# default number of filters is 4
def viz_layer(layer, n_filters= 4):
    fig = plt.figure(figsize=(20, 20))

    for i in range(n_filters):
        ax = fig.add_subplot(1, n_filters, i+1, xticks=[], yticks=[])
        # grab layer outputs
        ax.imshow(np.squeeze(layer[0,i].data.numpy()), cmap='gray')
        ax.set_title('Output %s' % str(i+1))

Regardons la sortie d’une couche convolutive, avant et après l’application d’une fonction d’activation ReLu.

In [ ]:
# plot original image
plt.imshow(gray_img, cmap='gray')

# visualize all filters
fig = plt.figure(figsize=(12, 6))
fig.subplots_adjust(left=0, right=1.5, bottom=0.8, top=1, hspace=0.05, wspace=0.05)
for i in range(4):
    ax = fig.add_subplot(1, 4, i+1, xticks=[], yticks=[])
    ax.imshow(filters[i], cmap='gray')
    ax.set_title('Filter %s' % str(i+1))


# convert the image into an input Tensor
gray_img_tensor = torch.from_numpy(gray_img).unsqueeze(0).unsqueeze(1)

# get the convolutional layer (pre and post activation)
conv_layer, activated_layer = model(gray_img_tensor)

# visualize the output of a conv layer
viz_layer(conv_layer)

#### Activation ReLu

Dans ce modèle, nous avons utilisé une fonction d'activation qui met à l'échelle la sortie de la couche convolutionnelle. Nous avons choisi une fonction ReLu pour ce faire, et cette fonction transforme simplement toutes les valeurs de pixels négatives en 0 (noir). Voir l'équation illustrée ci-dessous pour les valeurs de pixels d'entrée, `x`.

<img src='https://github.com/udacity/deep-learning-v2-pytorch/blob/master/convolutional-neural-networks/conv-visualization/notebook_ims/relu_ex.png?raw=1' height=50% width=50% />

In [ ]:
# Après que ReLU ait été activée
# voici la sortie d'une couche convolutionnelle après activation
viz_layer(activated_layer)